# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Config

In [14]:
from pathlib import Path
import pandas as pd
from scholarly import scholarly

In [23]:
pubs_csv_path = Path('./') / 'publications.csv'
pubs_md_dir = Path('../_publications/')
author_id = 'sh5mm9YAAAAJ'

## Import CSV

In [26]:
pubs_df = pd.read_csv(pubs_csv_path, header=0)
pubs_df

,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2009-10-01,Paper Title Number 1,Journal 1,This paper is about the number 1. The number 2...,"Your Name, You. (2009). ""Paper Title Number 1....",paper-title-number-1,http://academicpages.github.io/files/paper1.pdf
1,2010-10-01,Paper Title Number 2,Journal 1,This paper is about the number 2. The number 3...,"Your Name, You. (2010). ""Paper Title Number 2....",paper-title-number-2,http://academicpages.github.io/files/paper2.pdf
2,2015-10-01,Paper Title Number 3,Journal 1,This paper is about the number 3. The number 4...,"Your Name, You. (2015). ""Paper Title Number 3....",paper-title-number-3,http://academicpages.github.io/files/paper3.pdf


## Update from GG scholar 

In [27]:
author = scholarly.search_author_id(author_id)
author_lastname = author['name'].split(' ')[-1]

In [28]:
pubs_list = scholarly.fill(author, sections=['publications'])['publications']

In [32]:
for p in pubs_list:
    pub_title = p['bib']['title']
    nr_exist_matching = (pubs_df['title'] == pub_title).sum()
    if nr_exist_matching == 0:
        search_query = f"author:{author_lastname} {pub_title} {p['bib']['pub_year']}"
        new_pub = next(scholarly.search_pubs(search_query))
        
    elif nr_exist_matching > 1:
        print('Warning: there are some duplicate titles')

In [36]:
scholarly.bibtex(new_pub)

"@article{garnic2020dynamic,\n abstract = {The main purpose of this research is to assess the impact of aircraft noise around an airport considering spatial and temporal variations in the population. The concept of dynamic airport noise mapping has been demonstrated on Ljubljana airport case study. Detailed population mobility information for Slovenia were retrieved from a survey. The hourly noise contour levels generated by the airport's departure and arrival operations were calculated, and the annoyed population was thus estimated considering a reference scenario, where},\n author = {Garni{\\'c}, Emir and van Oosten, Nico and Meliveo, Luis and Jeram, Sonja and Louf, Thomas and Ramasco, Jose J},\n pub_year = {2020},\n title = {Dynamic noise maps for Ljubljana airport},\n venue = {NA}\n}\n\n"

In [35]:
scholarly.fill(new_pub)

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 1>,
 'bib': {'title': 'Dynamic noise maps for Ljubljana airport',
  'author': "Garni{\\'c}, Emir and van Oosten, Nico and Meliveo, Luis and Jeram, Sonja and Louf, Thomas and Ramasco, Jose J",
  'pub_year': '2020',
  'venue': 'NA',
  'abstract': "The main purpose of this research is to assess the impact of aircraft noise around an airport considering spatial and temporal variations in the population. The concept of dynamic airport noise mapping has been demonstrated on Ljubljana airport case study. Detailed population mobility information for Slovenia were retrieved from a survey. The hourly noise contour levels generated by the airport's departure and arrival operations were calculated, and the annoyed population was thus estimated considering a reference scenario, where",
  'pub_type': 'article',
  'bib_id': 'garnic2020dynamic'},
 'filled': True,
 'gsrank': 1,
 'pub_url': 'https://digital.csic.

In [25]:
pubs_list

[{'container_type': 'Publication',
  'source': <PublicationSource.AUTHOR_PUBLICATION_ENTRY: 2>,
  'bib': {'title': 'Dynamic noise maps for Ljubljana airport',
   'pub_year': '2020'},
  'filled': False,
  'author_pub_id': 'sh5mm9YAAAAJ:u5HHmVD_uO8C',
  'num_citations': 1}]

In [5]:
publications.to_csv('publications.csv', index=False)

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open(pubs_md_dir / md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md  2015-10-01-paper-title-number-3.md
2010-10-01-paper-title-number-2.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).